IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M5\FAULT_M5_10


In [3]:
guasto=2

VOLO m5 FAULT 10%

In [4]:
rcou_m5_fault10 = pd.read_csv("RCOU.csv")
rcou_m5_fault10 = rcou_m5_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m5_fault10 = rcou_m5_fault10[((rcou_m5_fault10['C9'] >= potenza) & (rcou_m5_fault10['C10'] >= potenza) & (rcou_m5_fault10['C11'] >= potenza) & (rcou_m5_fault10['C12'] >= potenza) & (rcou_m5_fault10['C13'] >= potenza) & (rcou_m5_fault10['C14']>= potenza))]
rcou_m5_fault10=rcou_m5_fault10.reset_index(drop=True)
display(rcou_m5_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,72720687,1542,1391,1459,1484,1528,1408
1,72731556,1415,1540,1494,1466,1428,1529
2,72741603,1472,1506,1414,1556,1393,1572
3,72752252,1524,1471,1426,1562,1454,1539
4,72762572,1495,1518,1476,1535,1461,1549
...,...,...,...,...,...,...,...
7390,166795551,1497,1409,1482,1427,1398,1507
7391,166805911,1488,1417,1464,1443,1369,1527
7392,166817138,1469,1437,1481,1424,1367,1528
7393,166827779,1454,1450,1494,1407,1369,1525


In [5]:
min=rcou_m5_fault10['TimeUS'][0]
max=rcou_m5_fault10['TimeUS'][len(rcou_m5_fault10)-1]
print(max)
print(min)

166914844
72720687


In [6]:
xkf1_m5_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m5_fault10 = xkf1_m5_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m5_fault10 = xkf1_m5_fault10[((xkf1_m5_fault10['TimeUS'] >= min) & (xkf1_m5_fault10['TimeUS'] <= max	))]
xkf1_m5_fault10 = xkf1_m5_fault10.reset_index(drop=True)
print(xkf1_m5_fault10)

         TimeUS  Roll  Pitch   Yaw
0      72729675 -0.65   0.55  2.33
1      72739678 -0.66   0.65  2.33
2      72750119 -0.61   0.61  2.33
3      72760283 -0.65   0.60  2.34
4      72770308 -0.62   0.57  2.37
...         ...   ...    ...   ...
8533  166869222  1.82  -2.80  4.06
8534  166879639  1.70  -2.83  4.07
8535  166890212  1.68  -2.82  4.08
8536  166900589  1.69  -2.85  4.07
8537  166912734  1.77  -2.79  4.08

[8538 rows x 4 columns]


In [7]:
att_m5_fault10 = pd.read_csv("ATT.csv")
att_m5_fault10 = att_m5_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m5_fault10 = att_m5_fault10[((att_m5_fault10['TimeUS'] >= min) & (att_m5_fault10['TimeUS'] <= max	))]
att_m5_fault10=att_m5_fault10.reset_index(drop=True)
print(att_m5_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0      72730714    -0.21 -0.65     -0.06   0.55    2.27  2.33
1      72740788    -0.22 -0.66     -0.06   0.65    2.27  2.33
2      72751532    -0.23 -0.61     -0.07   0.61    2.27  2.33
3      72761836    -0.22 -0.65     -0.07   0.60    2.27  2.34
4      72772125    -0.21 -0.62     -0.09   0.57    2.27  2.37
...         ...      ...   ...       ...    ...     ...   ...
8534  166870527     1.97  1.82      0.13  -2.80    5.21  4.06
8535  166881106     1.95  1.70      0.12  -2.83    5.21  4.07
8536  166891684     1.92  1.68      0.11  -2.82    5.21  4.08
8537  166902181     1.88  1.69      0.11  -2.85    5.21  4.07
8538  166914063     1.84  1.77      0.11  -2.79    5.21  4.08

[8539 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m5_fault10)>len(xkf1_m5_fault10):
    to_add=att_m5_fault10[len(xkf1_m5_fault10):]
    att_m5_fault10=att_m5_fault10[:len(xkf1_m5_fault10)]

if len(xkf1_m5_fault10)>len(att_m5_fault10):
    to_add=xkf1_m5_fault10[len(att_m5_fault10):]

for idx,i in enumerate(att_m5_fault10['Roll']):
    if(xkf1_m5_fault10['Roll'][idx] != i):
        att_m5_fault10['Roll'][idx] = ((att_m5_fault10['Roll'][idx] + xkf1_m5_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m5_fault10['Pitch']):
    if(xkf1_m5_fault10['Pitch'][idx] != i):
        att_m5_fault10['Pitch'][idx] = ((att_m5_fault10['Pitch'][idx] + xkf1_m5_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m5_fault10['Yaw']):
    if(xkf1_m5_fault10['Yaw'][idx] != i):
        att_m5_fault10['Yaw'][idx] = ((att_m5_fault10['Yaw'][idx] + xkf1_m5_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m5_fault10=pd.concat([att_m5_fault10,to_add])

print(att_m5_fault10)

C:\Users\chiar\AppData\Local\Temp\ipykernel_28308\4076637029.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m5_fault10['Roll'][idx] = ((att_m5_fault10['Roll'][idx] + xkf1_m5_fault10['Roll'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28308\4076637029.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m5_fault10['Pitch'][idx] = ((att_m5_fault10['Pitch'][idx] + xkf1_m5_fault10['Pitch'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28308\4076637029.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw
0      72730714    -0.21 -0.650     -0.06  0.550    2.27  2.330
1      72740788    -0.22 -0.660     -0.06  0.650    2.27  2.330
2      72751532    -0.23 -0.610     -0.07  0.610    2.27  2.330
3      72761836    -0.22 -0.650     -0.07  0.600    2.27  2.340
4      72772125    -0.21 -0.620     -0.09  0.570    2.27  2.370
...         ...      ...    ...       ...    ...     ...    ...
8534  166870527     1.97  1.760      0.13 -2.815    5.21  4.065
8535  166881106     1.95  1.690      0.12 -2.825    5.21  4.075
8536  166891684     1.92  1.685      0.11 -2.835    5.21  4.075
8537  166902181     1.88  1.730      0.11 -2.820    5.21  4.075
8538  166914063     1.84  1.770      0.11 -2.790    5.21  4.080

[8539 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m5_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m5_fault10 = esc_0_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m5_fault10 = esc_0_m5_fault10[((esc_0_m5_fault10['TimeUS'] >= min) & (esc_0_m5_fault10['TimeUS'] <= max))]
esc_0_m5_fault10=esc_0_m5_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m5_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m5_fault10 = esc_1_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m5_fault10 = esc_1_m5_fault10[((esc_1_m5_fault10['TimeUS'] >= min) & (esc_1_m5_fault10['TimeUS'] <= max))]
esc_1_m5_fault10=esc_1_m5_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m5_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m5_fault10 = esc_2_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m5_fault10 = esc_2_m5_fault10[((esc_2_m5_fault10['TimeUS'] >= min) & (esc_2_m5_fault10['TimeUS'] <= max))]
esc_2_m5_fault10=esc_2_m5_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m5_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m5_fault10 = esc_3_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m5_fault10 = esc_3_m5_fault10[((esc_3_m5_fault10['TimeUS'] >= min) & (esc_3_m5_fault10['TimeUS'] <= max))]
esc_3_m5_fault10=esc_3_m5_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m5_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m5_fault10 = esc_4_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m5_fault10 = esc_4_m5_fault10[((esc_4_m5_fault10['TimeUS'] >= min) & (esc_4_m5_fault10['TimeUS'] <= max))]
esc_4_m5_fault10=esc_4_m5_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m5_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m5_fault10 = esc_5_m5_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m5_fault10 = esc_5_m5_fault10[((esc_5_m5_fault10['TimeUS'] >= min) & (esc_5_m5_fault10['TimeUS'] <= max))]
esc_5_m5_fault10=esc_5_m5_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m5_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m5_fault10 = imu_0_m5_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m5_fault10 = imu_0_m5_fault10[((imu_0_m5_fault10['TimeUS'] >= min) & (imu_0_m5_fault10['TimeUS'] <= max))]

imu_1_m5_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m5_fault10 = imu_1_m5_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m5_fault10 = imu_1_m5_fault10[((imu_1_m5_fault10['TimeUS'] >= min) & (imu_1_m5_fault10['TimeUS'] <= max))]

imu_2_m5_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m5_fault10 = imu_2_m5_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m5_fault10 = imu_2_m5_fault10[((imu_2_m5_fault10['TimeUS'] >= min) & (imu_2_m5_fault10['TimeUS'] <= max))]

imu_m5_fault10 = pd.concat((imu_0_m5_fault10, imu_1_m5_fault10, imu_2_m5_fault10))
imu_m5_fault10=imu_m5_fault10.groupby(imu_m5_fault10.TimeUS, as_index=False).mean()

display(imu_m5_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,72725912,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562
1,72728103,-0.245029,0.217662,0.038900,-0.212036,0.573743,-9.924303
2,72730554,-0.146567,0.047290,0.027857,0.098784,0.691148,-9.809179
3,72733710,0.119616,-0.211923,-0.026932,0.110713,0.545332,-9.907626
4,72735559,0.112437,-0.099032,-0.026028,-0.145339,0.406057,-10.143177
...,...,...,...,...,...,...,...
32433,166897233,0.031076,0.046551,0.022947,-0.416947,0.473361,-9.839571
32434,166899201,0.032219,-0.082742,-0.006029,-0.362772,0.210693,-9.585236
32435,166902063,-0.058183,-0.035632,-0.009129,-0.587806,0.261268,-9.716467
32436,166904726,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m5_fault10 = pd.merge_ordered(imu_m5_fault10,att_m5_fault10)
m5_fault10=m5_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_0_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_1_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_2_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_3_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_4_m5_fault10)
m5_fault10 = pd.merge_ordered(m5_fault10, esc_5_m5_fault10)
m5_fault10=m5_fault10.fillna(method="ffill").fillna(method="bfill")
m5_fault10 = pd.merge_ordered(m5_fault10, rcou_m5_fault10)
m5_fault10=m5_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m5_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m5_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m5_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m5_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m5_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m5_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m5_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m5_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

8539
9348
10154
10956
11760
12546
13334


In [13]:
m5_fault10 = m5_fault10[~m5_fault10.TimeUS.isin(index_to_remove)]
m5_fault10["TimeUS"] = m5_fault10["TimeUS"] - m5_fault10.iloc[0]["TimeUS"]
m5_fault10["TimeUS"] = m5_fault10["TimeUS"].astype(int)
m5_fault10["Guasto"] = guasto

In [14]:
from datetime import timedelta
m5_fault10=m5_fault10.reset_index(drop=True)
m5_fault10['TimeUS'] = pd.to_datetime(m5_fault10['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m5_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.033646,0.092000,-0.004889,-0.370116,0.394487,-10.111562,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2
1,00:00:00.002191,-0.245029,0.217662,0.038900,-0.212036,0.573743,-9.924303,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2
2,00:00:00.004642,-0.146567,0.047290,0.027857,0.098784,0.691148,-9.809179,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1542.0,1391.0,1459.0,1484.0,1528.0,1408.0,2
3,00:00:00.007798,0.119616,-0.211923,-0.026932,0.110713,0.545332,-9.907626,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1415.0,1540.0,1494.0,1466.0,1428.0,1529.0,2
4,00:00:00.009647,0.112437,-0.099032,-0.026028,-0.145339,0.406057,-10.143177,-0.21,-0.650,-0.06,...,3.43,4091.0,3.82,1415.0,1540.0,1494.0,1466.0,1428.0,1529.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32433,00:01:34.171321,0.031076,0.046551,0.022947,-0.416947,0.473361,-9.839571,1.92,1.685,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2
32434,00:01:34.173289,0.032219,-0.082742,-0.006029,-0.362772,0.210693,-9.585236,1.92,1.685,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2
32435,00:01:34.176151,-0.058183,-0.035632,-0.009129,-0.587806,0.261268,-9.716467,1.92,1.685,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2
32436,00:01:34.178814,0.015743,0.110068,0.012201,-0.670795,0.566046,-10.021941,1.88,1.730,0.11,...,1.83,5126.0,2.33,1454.0,1450.0,1494.0,1407.0,1369.0,1525.0,2


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m5_fault10)/350)):
        V1=m5_fault10[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28308\361117201.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28308\361117201.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.053645,0.029699,-1.192426,0.180255,0.0,64.0,6.540367,4.542192,-0.010690,...,3375.276349,4.582934,0.267017,-0.354220,4.611892,77.0,75.0,32.823367,7.591723,2
1,1.0,-0.053473,0.029624,-1.190134,0.179999,0.0,81.0,6.911811,3.687883,-0.010745,...,3011.833791,4.586449,0.270604,-0.348371,4.615772,70.0,68.0,33.195043,6.799964,2
2,2.0,-0.053889,0.029601,-1.186900,0.180060,0.0,79.0,7.074568,3.145819,-0.010805,...,3097.008850,4.589943,0.274145,-0.344723,4.619626,79.0,77.0,33.488590,5.957339,2
3,3.0,-0.054693,0.029746,-1.180862,0.180702,0.0,62.0,7.098202,4.129917,-0.010568,...,3638.610931,4.593418,0.277643,-0.343042,4.623456,88.0,86.0,33.703032,5.090850,2
4,4.0,-0.054768,0.029664,-1.175161,0.180499,0.0,67.0,7.377486,4.769953,-0.010578,...,4110.769409,4.596873,0.281097,-0.343117,4.627261,75.0,73.0,33.837737,4.226383,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,87.0,-0.058300,0.025467,-0.911775,0.169729,0.0,89.0,25.546655,4.592043,-0.017839,...,9266.439954,4.889840,0.601185,-0.895352,4.950793,98.0,96.0,27.320973,10.414565,2
88,88.0,-0.058084,0.025430,-0.912665,0.169545,0.0,79.0,25.803229,3.995012,-0.017914,...,9311.247424,4.892961,0.604088,-0.905730,4.954168,100.0,98.0,27.658008,10.825209,2
89,89.0,-0.058175,0.025375,-0.907375,0.169417,0.0,87.0,25.923531,3.360682,-0.018139,...,9316.418082,4.896068,0.606960,-0.915998,4.957526,114.0,111.0,27.963692,11.085448,2
90,90.0,-0.058445,0.025350,-0.903968,0.169435,0.0,90.0,25.963259,4.336847,-0.018040,...,9282.331347,4.899161,0.609799,-0.926155,4.960866,106.0,103.0,28.237148,11.192656,2


In [18]:
path_file = path_file.replace(r"M5\FAULT_M5_10", "")
os.chdir(path_file)
df_merged.to_csv('m5_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
